# Preprocess data for machine learning

### March 3, 2018
### Tiffany Huang

In [164]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

# Input data files are available in the "WDataFiles/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "WDataFiles"]).decode("utf8"))

WCities.csv
WFieldGoalPercentage.csv
WFieldGoalPercentage_IDs.csv
WGameCities.csv
WNCAATourneyCompactResults.csv
WNCAATourneySeeds.csv
WNCAATourneySlots.csv
WReboundMargin.csv
WReboundMargin_IDs.csv
WRegularSeasonCompactResults.csv
WSampleSubmissionStage1.csv
WScoringMargin.csv
WScoringMargin_IDs.csv
WScoringOffense.csv
WScoringOffense_IDs.csv
WSeasons.csv
WTeamSpellings.csv
WTeams.csv
WTurnovers.csv
WTurnovers_IDs.csv
ranking_macro.xlsm



# Load the Training Data

In [180]:
data_dir = 'WDataFiles/'
df_seeds = pd.read_csv(data_dir + 'WNCAATourneySeeds.csv')
df_tourney = pd.read_csv(data_dir + 'WNCAATourneyCompactResults.csv')
df_reg_season = pd.read_csv(data_dir + 'WRegularSeasonCompactResults.csv')
df_field_goal = pd.read_csv(data_dir + 'WFieldGoalPercentage_IDs.csv')
df_rebounds = pd.read_csv(data_dir + 'WReboundMargin_IDs.csv')
df_scoring_margin = pd.read_csv(data_dir + 'WScoringMargin_IDs.csv')
df_scoring_offense = pd.read_csv(data_dir + 'WScoringOffense_IDs.csv')
df_turnovers = pd.read_csv(data_dir + 'WTurnovers_IDs.csv')

# Seeds Data

In [181]:
df_seeds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [182]:
# First, we'll simplify the datasets to remove the columns we won't be using 
# and convert the seedings to the needed format (stripping the regional abbreviation in front of the seed).

# Get just the digits from the seeding. Return as int
def seed_to_int(seed):
    s_int = int(seed[1:3])
    return s_int

df_seeds.drop(df_seeds[df_seeds.Season < 2002].index, inplace=True)
df_seeds.index = range(len(df_seeds))
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
df_seeds.head()

,Season,TeamID,seed_int
0,2002,3181,1
1,2002,3124,2
2,2002,3376,3
3,2002,3400,4
4,2002,3256,5


# Tournament Data

In [183]:
df_tourney.drop(labels=['DayNum', 'WScore', 'LScore', 'NumOT'], inplace=True, axis=1)
df_tourney.drop(df_tourney[df_tourney.Season < 2002].index, inplace=True)
df_tourney.index = range(len(df_tourney))
df_tourney.head()

,Season,WTeamID,LTeamID,WLoc
0,2002,3116,3155,H
1,2002,3124,3137,H
2,2002,3160,3380,H
3,2002,3179,3393,H
4,2002,3181,3313,H


In [184]:
# Merge the Seeds with their corresponding TeamIDs in the compact results dataframe.
df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tourney, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat.drop(labels=['WSeed', 'LSeed'], inplace=True, axis=1)
df_concat.head()

,Season,WTeamID,LTeamID,WLoc,SeedDiff
0,2002,3116,3155,H,-5
1,2002,3124,3137,H,-13
2,2002,3160,3380,H,-11
3,2002,3179,3393,H,-3
4,2002,3181,3313,H,-15


# Regular Season Data

In [185]:
df_reg_season.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_reg_season.drop(df_reg_season[df_reg_season.Season < 2002].index, inplace=True)
df_reg_season.index = range(len(df_reg_season))
df_reg_season.head()

,Season,WTeamID,LTeamID
0,2002,3160,3331
1,2002,3163,3193
2,2002,3191,3396
3,2002,3198,3372
4,2002,3307,3334


In [186]:
# Find the record during the regular season between two teams
def regSeasonToRecord(df):
    record_dict = {}
    for row in df.itertuples():
        if (row[1] not in record_dict):
            record_dict[row[1]] = {}
        else:
            if ((row[2], row[3]) in record_dict[row[1]]):
                record_dict[row[1]][(row[2], row[3])] += 1
            else:
                record_dict[row[1]][(row[2], row[3])] = 1
    return record_dict

record_dict = regSeasonToRecord(df_reg_season)

In [187]:
# Add the record between the winning and losing teams to the table
# Iterate through df_concat rows, look for record in dictionary between WTeam and LTeam and add it to the row
df_record = pd.DataFrame()
for row in df_concat.itertuples():
    if ((row[2], row[3]) in record_dict[row[1]]):
        df_record = df_record.append(pd.DataFrame([record_dict[row[1]][(row[2], row[3])]], columns=['Record']), ignore_index=True)
    elif ((row[3], row[2]) in record_dict[row[1]]):
        df_record = df_record.append(pd.DataFrame([-1.0 * record_dict[row[1]][(row[3], row[2])]], columns=['Record']), ignore_index=True)
    else:
        df_record = df_record.append(pd.DataFrame([0], columns=['Record']), ignore_index=True)
        
df_concat = pd.concat([df_concat, df_record], axis=1)
df_concat

,Season,WTeamID,LTeamID,WLoc,SeedDiff,Record
0,2002,3116,3155,H,-5,0.0
1,2002,3124,3137,H,-13,0.0
2,2002,3160,3380,H,-11,0.0
3,2002,3179,3393,H,-3,0.0
4,2002,3181,3313,H,-15,0.0
5,2002,3198,3166,H,-7,0.0
6,2002,3243,3245,H,-11,0.0
7,2002,3261,3365,H,-5,0.0
8,2002,3280,3130,A,7,0.0
9,2002,3323,3307,H,-3,0.0


# Field Goal Percentage Data

In [188]:
df_field_goal.drop(labels=['Team', 'Kaggle Name', 'Ranking'], inplace=True, axis=1)
df_field_goal.drop(df_field_goal[df_field_goal.Season > 2017].index, inplace=True)
df_field_goal['FG Percentage'] = df_field_goal['FG Percentage'] / 100
df_field_goal['Kaggle TeamID'] = df_field_goal['Kaggle TeamID'].astype(int)
df_field_goal.head()
print(df_field_goal[(df_field_goal.Season == 2014) & (df_field_goal['Kaggle TeamID'] == 3119)])

      Season  FG Percentage  Kaggle TeamID
4066    2014          0.411           3119


In [189]:
df_win_fg = df_field_goal.rename(columns={'Kaggle TeamID':'WTeamID', 'FG Percentage':'WFGP'})
df_win_fg.head()
df_lose_fg = df_field_goal.rename(columns={'Kaggle TeamID':'LTeamID', 'FG Percentage':'LFGP'})
df_concat = pd.merge(left=df_concat, right=df_win_fg, how='inner', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_concat, right=df_lose_fg, how='inner', on=['Season', 'LTeamID'])
df_concat['FG%Diff'] = df_concat.WFGP - df_concat.LFGP
df_concat.drop(labels=['WFGP', 'LFGP'], inplace=True, axis=1)

In [190]:
# Now we'll create a dataframe that summarizes wins & losses along with 
# their corresponding seed differences. This is the meat of what we'll be creating our model on.
df_wins = pd.DataFrame()
df_wins = df_concat.drop(labels=['Season', 'WTeamID', 'LTeamID', 'WLoc'], axis=1)
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['Record'] = -df_concat['Record']
df_losses['FG%Diff'] = -df_concat['FG%Diff']
df_losses['Result'] = 0

df_predictions = pd.concat((df_wins, df_losses))
df_predictions = (df_predictions - df_predictions.min()) / (df_predictions.max() - df_predictions.min())
df_predictions

,SeedDiff,Record,FG%Diff,Result
0,0.333333,0.500000,0.553797,1.0
1,0.066667,0.500000,0.518987,1.0
2,0.133333,0.500000,0.639241,1.0
3,0.400000,0.666667,0.484177,1.0
4,0.533333,0.500000,0.433544,1.0
5,0.400000,0.500000,0.610759,1.0
6,0.666667,0.500000,0.518987,1.0
7,0.000000,0.500000,0.867089,1.0
8,0.266667,0.500000,0.731013,1.0
9,0.400000,0.500000,0.705696,1.0


In [191]:
X_train = df_predictions[['SeedDiff', 'FG%Diff']].values.reshape(-1,2)
y_train = df_predictions.Result.values
X_train, y_train = shuffle(X_train, y_train)

# Train the Model

In [192]:
# Use a basic logistic regression to train the model. You can set different C values to see how performance changes.
logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=9)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

Best log_loss: -0.4434, with best C: 100.0


In [193]:
X = np.arange(-10, 10).reshape(-1, 1)
preds = clf.predict_proba(X)[:,1]

plt.figure()
plt.plot(X, preds)
plt.xlabel('Team1 seed - Team2 seed')
plt.ylabel('P(Team1 will win)')

ValueError: X has 1 features per sample; expecting 2

In [194]:
df_sample_sub = pd.read_csv(data_dir + 'WSampleSubmissionStage1.csv')
n_test_games = len(df_sample_sub)
def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

In [199]:
from sklearn.preprocessing import MinMaxScaler

X_test = np.zeros(shape=(n_test_games, 2))
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    diff_seed = t1_seed - t2_seed
    X_test[ii, 0] = diff_seed
    
    t1_fgp = df_field_goal[(df_field_goal['Kaggle TeamID'] == t1) & (df_field_goal.Season == year)]['FG Percentage'].values[0]
    t2_fgp = df_field_goal[(df_field_goal['Kaggle TeamID'] == t2) & (df_field_goal.Season == year)]['FG Percentage'].values[0]
    diff_fgp = t1_fgp - t2_fgp
    X_test[ii, 1] = diff_fgp

scaler = MinMaxScaler()
scaler.fit(X_test)
X_test_normalized = scaler.transform(X_test)

In [205]:
preds = clf.predict_proba(X_test_normalized)[:,1]

clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = clipped_preds
df_sample_sub

,ID,Pred
0,2014_3103_3107,0.558566
1,2014_3103_3113,0.312261
2,2014_3103_3119,0.570564
3,2014_3103_3124,0.063559
4,2014_3103_3140,0.455601
5,2014_3103_3143,0.252510
6,2014_3103_3151,0.372921
7,2014_3103_3163,0.050000
8,2014_3103_3169,0.764926
9,2014_3103_3173,0.145365


In [206]:
# Create submission file!
df_sample_sub.to_csv('field_goal.csv', index=False)